In [8]:
%run ./libs.ipynb
%run ./initialize.ipynb
%run ./signals_sec321.ipynb

AutoPhase = True
noise = 0.12

In [31]:
def getPlot(Ss, dsampled, Sfilt, PhySig, Noversampl, BASE, SAMP, SampSig):
    rf_vec1 = dsampled/(LB/2) #np.linspace(-1,1,len(dsampled))
    rf_vec2 = np.linspace(-1000,1000,len(Sfilt))
    
    fig = plt.figure(figsize = (15,4))
    ax = fig.add_subplot(111)
    
    if SampSig:
        line1 = ax.plot(rf_vec1, 10*np.log10(Ss),'r', linewidth='2')
    
        if PhySig:
            line2 = ax.plot(rf_vec2[int(len(Sfilt)/2-len(Sfilt)/1000/2*5):int(len(Sfilt)/2-len(Sfilt)/1000/2*Noversampl)],
                        10*np.log10(Sfilt[int(len(Sfilt)/2-len(Sfilt)/1000/2*5):int(len(Sfilt)/2-len(Sfilt)/1000/2*Noversampl)]),'k', linewidth='1')

            line3 = ax.plot(rf_vec2[int(len(Sfilt)/2+len(Sfilt)/1000/2*Noversampl):int(len(Sfilt)/2+len(Sfilt)/1000/2*5)],
                        10*np.log10(Sfilt[int(len(Sfilt)/2+len(Sfilt)/1000/2*Noversampl):int(len(Sfilt)/2+len(Sfilt)/1000/2*5)]),'k', linewidth='1')

    else:
        if PhySig:
            line2 = ax.plot(rf_vec2, 10*np.log10(Sfilt),'k', linewidth='1')
        
    if SAMP:
        SAMPrange = rf_vec1 + Noversampl
        LB_rect = np.heaviside(SAMPrange,0.5)
        LB_rect[rf_vec1 > Noversampl] = 0
        trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.fill_between(rf_vec1, 0, 1, where=LB_rect > 0.5,
                        facecolor='red', alpha=0.15, transform=trans)
        
    if BASE:
        LBrange = rf_vec1 + 1
        LB_rect = np.heaviside(LBrange,0.5)
        LB_rect[rf_vec1 > 1] = 0
        trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
        ax.fill_between(rf_vec1, 0, 1, where=LB_rect > 0.5,
                        facecolor='blue', alpha=0.15, transform=trans)
        
    plt.ylabel('Signal, a.u.', fontsize = 15)
    plt.xlabel('f/fo', fontsize = 15)
    plt.ylim([-55, 2])
    plt.xlim([-5, 5])
    plt.xticks(np.arange(-5, 5, step=1))
    plt.grid(color='k', linestyle='-', linewidth=.1)
    plt.show()         
    
def getSig(Nsampl, LPF, order, LPFtype, f_slide, Filt_slider, PhySig_box, BASE_box, SAMP_box, SampSig_box):

    SampSig = SampSig_box
    SAMP = SAMP_box
    BASE = BASE_box
    PhySig = PhySig_box
    FILTFACT = Filt_slider
    f = f_slide
    FiltType = LPFtype
    Order = order
    FILT = LPF
    Noversampl = Nsampl
    depth0 = f*LB/2*1e6
    
    Ss, dsampled, Sfilt = signals_sec321(Noversampl, depth0, oof, ofcont, FILT, Order, FiltType, FILTFACT, AutoPhase, ph0, IQ, noise, PhySig)
    Ss = Ss/3.11e-7
    Sfilt = Sfilt/3.11e-3
    getPlot(Ss, dsampled, Sfilt, PhySig, Noversampl, BASE, SAMP, SampSig)

In [32]:
# depth
f_slide = widgets.FloatSlider(value = .5,
                              description = 'f/fo',
                              min = -2,
                              max = 2,
                              step = .1,
                              continuous_update = True,
                              layout=Layout(width='430px', margin='14px 0px'))

Filt_slider = widgets.FloatSlider(value = 4,
                                  min=0.7, max=5, step=0.1,
                                  description = 'LPF cutoff',
                                  readout_format='.1f',
                                  layout=Layout(width='430px', margin='0px 0px'))
# N oversampling
Nsampl = widgets.FloatText(value = 2.7,
                           description = 'Oversampl.',
                           min = 2,
                           max = 5,                         
                           layout=Layout(width='150px'))

# low-pass filter
LPF = widgets.ToggleButton(value = True,
                           description = 'LPF',
                           layout=Layout(width='132px', margin='1px 90px')) 

order = widgets.Dropdown(options = [('2',1),('3',2),('4',3),('5',4)],
                         value = 2,
                         description = 'Order:',
                         layout=Layout(width='220px'))

LPFtype = widgets.Dropdown(options = [('Butter', 1), ('Cheby', 2)],
                           value = 1,
                           description = 'Type:',
                           layout=Layout(width='220px'))


PhySig_box = widgets.Checkbox(value=True, description='Physical signal', disabled=False, indent=False)

SampSig_box = widgets.Checkbox(value=True, description='Sampled signal', disabled=False, indent=False)

SAMP_box = widgets.Checkbox(value=False, description='Sampling window', disabled=False, indent=False)

BASE_box = widgets.Checkbox(value=True, description='Baseband window', disabled=False, indent=False)

In [ ]:
ui1 = widgets.VBox([Nsampl])
ui2 = widgets.VBox([LPF, order, LPFtype])
ui3 = widgets.VBox([f_slide, Filt_slider])
ui4 = widgets.HBox([ui1, ui2, ui3])
out = widgets.interactive_output(getSig, {'Nsampl': Nsampl, 'LPF': LPF, 'order': order, 
                                          'LPFtype': LPFtype, 'f_slide': f_slide, 'Filt_slider': Filt_slider,
                                          'PhySig_box': PhySig_box, 'SampSig_box': SampSig_box, 'BASE_box': BASE_box, 'SAMP_box': SAMP_box})
display(ui4, out)

ui5 = widgets.HBox([PhySig_box, SampSig_box, SAMP_box, BASE_box])
display(ui5)